## 5.5 MyFinance

Aplikacja MyFinance wykorzystuje `TabLayout Navigation` z `ViewPager2`. Dane dostarczymy jak w poprzednich przykładach przez obiekt `dataProvider` zawierający hardcodowane przykładowe pozycje. Aplikacja zawiera trzy ekrany pokazujące stan oszczędności oraz rachunki - ekran główny zawiera podsumowanie. Dodamy customową animację do `TabLayout.Tab` oraz `DonutChart` podsumowujący stan naszych finansów oraz kilka elementów urozmaicających wygląd naszej aplikacji. Sam layout aplikacji bazujev na przykładzie z **Android Open Project** [link](https://github.com/android/compose-samples/tree/main/Rally)

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

Nasza aplikacja będzie posiadała zdefiniowany layout tylko w orientacji `portrait`, więc zablookujemy możliwość zmiany w plik `AndroidManifest.xml` - do aktywności dodamy

```xml
android:screenOrientation="portrait"
```

### **TabLayout i ViewPager2**

Aplikację rozpoczniemy od dodania nawigacji. Dodajmy trzy fragmenty (`BlankFragment`) - `OverviewFragment`, `AccountsFragment`, `BillsFragment`

In [ ]:
class AccountsFragment : Fragment() {
    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_accounts, container, false)
    }
}

class BillsFragment : Fragment() {
    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_bills, container, false)
    }
}

class OverviewFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_overview, container, false)
    }
}

W layoutach chwilowo zmieniam tylko text pola `TextView` na nazwę fragmentu. Ponieważ nasza aplikacja będzie posiadać tylko jeden motyw usuwamy plik `themes.xml(night)` do pliku `colors.xml` dodajemy kilka kolorów

In [ ]:
<color name="green_500">#FF1EB980</color>
<color name="dark_blue_900">#FF26282F</color>
<color name="dark_blue_500">#FF004940</color>

Następnie modyfikujemy plik `themes.xml`

In [ ]:
<style name="Theme.MyFinanceAppKotlin" parent="Theme.MaterialComponents.DayNight.NoActionBar">
    <!-- Primary brand color. -->
    <item name="colorPrimary">@color/dark_blue_500</item>
    <item name="colorPrimaryVariant">@color/dark_blue_900</item>
    <item name="colorOnPrimary">@color/teal_700</item>
    <!-- Secondary brand color. -->
    <item name="colorSecondary">@color/teal_700</item>
    <item name="colorSecondaryVariant">@color/teal_200</item>
    <item name="colorOnSecondary">@color/white</item>
    <!-- Status bar color. -->
    <item name="android:statusBarColor" tools:targetApi="l">?attr/colorPrimaryVariant</item>
    <!-- Customize your theme here. -->
</style>

Aplikacja nie będzie wykorzystywać `ActionBar`, więc jako `parent` podajemy `NoActionBar`.

Nasz `ViewPager2` dodamy bezpośrednio do `MainActivity` - zmodyfikujmy layout

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <com.google.android.material.tabs.TabLayout
        android:id="@+id/tabLayout"
        android:layout_width="match_parent"
        android:background="@color/dark_blue_900"
        app:tabTextColor="@color/white"
        android:layout_height="wrap_content"
        app:tabInlineLabel="true"
        app:tabIconTint="@color/teal_200"
        app:tabMode="fixed"
        app:tabGravity="start"
        app:tabMaxWidth="0dp" />

    <androidx.viewpager2.widget.ViewPager2
        android:id="@+id/viewPager"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"/>

</LinearLayout>

Kilka elementów zastosowanych w `TabLayout`
- `tabMode` - sposób w jaki mają zostać wyświetlone zakładki
    - `fixed` - tak jak zostaną podane, rozmiar zostanie automatycznie dostosowany
    - `scrollable` - zakładki nie będą dokowane na całą szerokość ekranu, tylko kilka zostanie wyświetlonych - dostęp do niewidocznych użytkownik może uzyskać przez przewijanie
- `tabGravity` - sposób rozmieszczenia zakładek
- `tabMaxWidth` - ustawienie maksymalnej szerokości - `0dp` dostosowuje rozmiar zakładki do jej zawartości
- `tabIconTint` - kolor wyświetlanej ikony
- `tabInlineLabel` - ustawienie ikony i tekstu w rzędzie - przyjmuje `true` lub `false`

`ViewPager2` zawiera `layout_height` ustawiony na `0dp` oraz `layout_weight = "1"` pozwala wypełnić pozostałą przestrzeń w `LinearLayout`

Do pliku `build.gradle(Module)` w bloku `dependencies` dodajmy

In [ ]:
implementation "androidx.viewpager2:viewpager2:1.0.0"

Po zsynchronizowaniu projektu dodajmy klasę `FinanceAdapter` do pliku `MainActivity`

In [ ]:
class FinanceAdapter(fragmentActivity: FragmentActivity) 
    : FragmentStateAdapter(fragmentActivity) {
    private val fragments = listOf(
        OverviewFragment(), 
        AccountsFragment(), 
        BillsFragment())
    override fun getItemCount(): Int = fragments.size
    override fun createFragment(position: Int): Fragment = fragments[position]
}

Następnie utworzymy `ViewPager2` i `TabLayout` w metodzie `onCreate` głównej aktywności

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_main)

    val viewPager2 = findViewById<ViewPager2>(R.id.viewPager)
    viewPager2.apply {
        adapter = FinanceAdapter(this@MainActivity)
    }

    val tabLayout: TabLayout = findViewById(R.id.tabLayout)
    tabLayoutSetup(this, tabLayout, viewPager2)
}

Następnym krokiem jest napisanie metody `tabLayoutSetup`, dodajmy do projektu nowy pakiet `util` i w nim utworzymy nowy plik `UiSetup`. Do pliku dodajmy tytuły naszych zakładek

In [ ]:
private val tabTitles = listOf(
    R.string.overview_name, 
    R.string.accounts_name, 
    R.string.bills_name)

I do pliku `strings.xml`

In [ ]:
<string name="overview_name">Overview</string>
<string name="accounts_name">Accounts</string>
<string name="bills_name">Bills</string>

Dodajmy metodę `tabLayoutSetup` do pliku `UiSetup`

In [ ]:
fun tabLayoutSetup(context: Context, tabLayout: TabLayout, viewPager2: ViewPager2) {
    setupTabLayoutMediator(context, tabLayout, viewPager2)
}

private fun setupTabLayoutMediator(
    context: Context, 
    tabLayout: TabLayout, 
    viewPager2: ViewPager2) {
    TabLayoutMediator(tabLayout, viewPager2) { tab, position ->
        tab.apply {text = context.getString(tabTitles[position])}
    }.attach()
}

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/9IdIFNbyyewccfTTjG/giphy.gif?cid=790b76113f11b618eb6ce26cdf2e17faf07d98203cdf7e21&rid=giphy.gif&ct=g" width="150" />

Zmodyfikujmy metodę `tabLayoutSetup`

In [ ]:
private val tabTitles = listOf(
    R.string.overview_name, 
    R.string.accounts_name,
    R.string.bills_name)
private val tabIcons = listOf(
    R.drawable.ic_overview, 
    R.drawable.ic_accounts, 
    R.drawable.ic_bills)

fun tabLayoutSetup(context: Context, tabLayout: TabLayout, viewPager2: ViewPager2) {
    setupTabLayoutMediator(context, tabLayout, viewPager2)
}

private fun setupTabLayoutMediator(
    context: Context, 
    tabLayout: TabLayout, 
    viewPager2: ViewPager2) {
    TabLayoutMediator(tabLayout, viewPager2) { tab, position ->
        tab.icon = ContextCompat.getDrawable(context, tabIcons[position])
        tab.apply {
            text = context.getString(tabTitles[position])
            icon?.setTint(Color.WHITE)
        }
    }.attach()
}

Metoda `icon?.setTint` pozwala zmienić kolor grafiki wektorowej - ustawiamy na białą. Nasza aplikacja teraz wygląda następująco.

<img src="https://media0.giphy.com/media/gGFILN8lnvSsCIacNt/giphy.gif?cid=790b7611d2de0256b614247faddd193614ba76fa00201eec&rid=giphy.gif&ct=g" width="150" />

Chcemy pokazać tylko ikonę nie zaznaczonej zakładki oraz ikonę z tekstem wybranej zakładce. Dodajmy metodę `tabSelection` w której zaimplementujemy metody `addOnTabSelectedListener`. 

`TabSelectionListener` obsługuje tylko zdarzenia `onTabSelected`, `onTabUnselected` i `onTabReselected`. W pierwszych dwóch zmienimy tekst zakładki

In [ ]:
private val tabTitles = listOf(
    R.string.overview_name, 
    R.string.accounts_name, 
    R.string.bills_name)
private val tabIcons = listOf(
    R.drawable.ic_overview, 
    R.drawable.ic_accounts, 
    R.drawable.ic_bills)

fun tabLayoutSetup(context: Context, tabLayout: TabLayout, viewPager2: ViewPager2) {
    setupTabLayoutMediator(context, tabLayout, viewPager2)
    setupTabSelection(context, tabLayout)
}

private fun setupTabLayoutMediator(
    context: Context, 
    tabLayout: TabLayout, 
    viewPager2: ViewPager2) {
    TabLayoutMediator(tabLayout, viewPager2) { tab, position ->
        tab.icon = ContextCompat.getDrawable(context, tabIcons[position])
        tab.apply {
            text = context.getString(tabTitles[position])
            icon?.setTint(Color.WHITE)
        }
    }.attach()
}

private fun setupTabSelection(context: Context, tabLayout: TabLayout){
    tabLayout.addOnTabSelectedListener(object : TabLayout.OnTabSelectedListener {
        override fun onTabSelected(tab: TabLayout.Tab) {
            tab.apply {
                text = context.getString(tabTitles[tab.position])
            }
        }

        override fun onTabUnselected(tab: TabLayout.Tab) {
            tab.text = ""
        }

        override fun onTabReselected(tab: TabLayout.Tab) {}
    })
}

<img src="https://media0.giphy.com/media/uzc9KIxaD4KinhS1GQ/giphy.gif?cid=790b7611153c63bbec947cab1bc09d6c638f7323bdc36e8c&rid=giphy.gif&ct=g" width="150" />

Musimy jeszcze zainicjować w odpowiedni sposób zakładki. Zmodyfikujmy metodę `setupTabLayoutMediator`. Tekst i kolor ikony ustawimy tylko dla `position == 0`, co odpowiada ekranowi głównemu aplikacji.

In [ ]:
private fun setupTabLayoutMediator(
    context: Context, 
    tabLayout: TabLayout, 
    viewPager2: ViewPager2) {
    TabLayoutMediator(tabLayout, viewPager2) { tab, position ->
        tab.icon = ContextCompat.getDrawable(context, tabIcons[position])
        if (position == 0) tab.apply {
            text = context.getString(tabTitles[position])
            icon?.setTint(Color.WHITE)
        }
    }.attach()
}

<img src="https://media0.giphy.com/media/8wosma6sipcea16wxt/giphy.gif?cid=790b76114eedb95754c5e7f3b59ff15167d82c6bc0910831&rid=giphy.gif&ct=g" width="150" />

Dodajmy customową animację do zakładek. Użyjemy prostego skalowania - niezaznaczona zakładka będzie wyświetlać tylko przeskalowaną (70%) ikonę. Dodajmy metodę `initialTabsSetup`

In [ ]:
private fun initialTabsSetup(viewGroup: ViewGroup, tabLayout: TabLayout){
    for (i in 0 until viewGroup.childCount) {
        if (i == tabLayout.selectedTabPosition) continue
        val tab = viewGroup.getChildAt(i)
        tab.scaleX = tabScaleLow
        tab.scaleY = tabScaleLow
    }
}

Skalować możemy tylko na obiektach `View` - bezpośrednio nie możemy tego zrobić na `TabLayout.Tab`, ponieważ nie rozszerza on `View`. Więc jako parametr metody podamy `ViewGroup`, który będzie naszym zawierał nasze zakładki. Chcemy skalować wszystkie elementy poza aktualnie zaznaczonym, dlatego mamy warunek `if (...) continue`. Następnie wywołajmy tą metodę w `tabLayoutSetup`

In [ ]:
fun tabLayoutSetup(context: Context, tabLayout: TabLayout, viewPager2: ViewPager2) {
    setupTabLayoutMediator(context, tabLayout, viewPager2)
    val vg = tabLayout.getChildAt(0) as ViewGroup
    initialTabsSetup(vg, tabLayout)
    setupTabSelection(context, tabLayout)
}

Z `TabLayout` wywołujemy metodę `getChildAt()` wyciągając wszyskie `Tabs` przepisane do tego `TabLayout` i rzutujemy na `ViewGroup` - co pozwala użyć na `Tabs` animacji. Dodajmy metodę `setupAnimation` wykonującą prostą animację powiększania.

In [ ]:
private fun setupAnimation(view: View, scale: Float, duration: Long){
    view.animate()
        .scaleX(scale)
        .scaleY(scale)
        .setInterpolator(FastOutSlowInInterpolator())
        .setDuration(duration)
        .start()
}

- `scaleX`, `scaleY` - skalowanie do zadanej wartości z przedziału (0.0 - 1.0)
- `setInterpolator` - umożliwia wykorzystanie podstawowych animacji
- `FastOutSlowInInterpolator` - wykorzysuje [tablicowanie](https://pl.wikipedia.org/wiki/Tablicowanie) z [krzywymi Beziera](https://pl.wikipedia.org/wiki/Krzywa_B%C3%A9ziera) do wykonania utworzenia animacji - tutaj zastosujemy skalowanie
- `setDuration` - długość animacji w milisekundach

Teraz wykorzystajmy tą metodę do zmiany rozmiarów zakładek po zaznaczeniu.

In [ ]:
private const val tabScaleLow = 0.7f
private const val tabScaleHigh = 1f

private val tabTitles = listOf(
    R.string.overview_name, 
    R.string.accounts_name, 
    R.string.bills_name)
private val tabIcons = listOf(
    R.drawable.ic_overview, 
    R.drawable.ic_accounts, 
    R.drawable.ic_bills)

fun tabLayoutSetup(context: Context, tabLayout: TabLayout, viewPager2: ViewPager2) {
    setupTabLayoutMediator(context, tabLayout, viewPager2)
    val vg = tabLayout.getChildAt(0) as ViewGroup
    initialTabsSetup(vg, tabLayout)
    setupTabSelection(context, vg, tabLayout)
}

private fun setupTabLayoutMediator(
    context: Context, 
    tabLayout: TabLayout, 
    viewPager2: ViewPager2) {
    TabLayoutMediator(tabLayout, viewPager2) { tab, position ->
        tab.icon = ContextCompat.getDrawable(context, tabIcons[position])
        if (position == 0) tab.apply {
            text = context.getString(tabTitles[position])
            icon?.setTint(Color.WHITE)
        }
    }.attach()
}

private fun initialTabsSetup(viewGroup: ViewGroup, tabLayout: TabLayout){
    for (i in 0 until viewGroup.childCount) {
        if (i == tabLayout.selectedTabPosition) continue
        val a = viewGroup.getChildAt(i)
        a.scaleX = tabScaleLow
        a.scaleY = tabScaleLow
    }
}

private fun setupTabSelection(context: Context, viewGroup: ViewGroup, tabLayout: TabLayout){
    tabLayout.addOnTabSelectedListener(object : TabLayout.OnTabSelectedListener {
        override fun onTabSelected(tab: TabLayout.Tab) {
            tab.apply {
                text = context.getString(tabTitles[tab.position])
                icon?.setTint(Color.WHITE)
            }

            setupAnimation(viewGroup.getChildAt(tab.position), tabScaleHigh,
                context.resources.getInteger(android.R.integer.config_mediumAnimTime).toLong())
        }

        override fun onTabUnselected(tab: TabLayout.Tab) {
            tab.text = ""
            setupAnimation(viewGroup.getChildAt(tab.position), tabScaleLow, 1L)
        }

        override fun onTabReselected(tab: TabLayout.Tab) {}
    })
}

private fun setupAnimation(view: View, scale: Float, duration: Long){
    view.animate()
        .scaleX(scale)
        .scaleY(scale)
        .setInterpolator(FastOutSlowInInterpolator())
        .setDuration(duration)
        .start()
}

<img src="https://media2.giphy.com/media/4sMOjoEDLmVGtiC7FD/giphy.gif?cid=790b761140e3aa2701d05ed0f4d943a6f3bf656a58a60e90&rid=giphy.gif&ct=g" width="150" />

### **Dane**

Dodajmy dane do naszego projektu

In [ ]:
data class Account(
    val name: String,
    val number: String,
    val amount: Double,
    val color: Int
)

In [ ]:
import java.time.LocalDate

data class Bill (
    val name: String,
    val endDate: LocalDate,
    val amount: Double,
    val color: Int
)

In [ ]:
import android.graphics.Color
import java.time.LocalDate

object DataProvider {

    val accounts = listOf(
        Account("Home savings", "1111111111", 23456.34, Color.BLUE),
        Account("Car savings", "2222222222", 126578.99, Color.LTGRAY),
        Account("Vacation", "3457733323", 9875.12, Color.MAGENTA),
        Account("Emergency", "9488344443", 10000.77, Color.RED),
        Account("Healthcare", "3243554434", 12345.00, Color.YELLOW),
        Account("Shopping", "2947560007", 3456.56, Color.BLACK)
    )

    val bills = listOf(
        Bill("Bank Credit", LocalDate.of(2022, 9,22), 2300.0, Color.BLACK),
        Bill("Tuition", LocalDate.of(2023, 2,10), 1200.0, Color.BLUE),
        Bill("Rent", LocalDate.of(2022, 8,3), 1023.87, Color.YELLOW),
        Bill("Loan", LocalDate.of(2022, 12,22), 334.0, Color.GRAY),
        Bill("Car Repair", LocalDate.of(2023, 1,9), 982.38, Color.WHITE),
        Bill("Dress Loan", LocalDate.of(2023, 5,18), 243.0, Color.MAGENTA)
    )

    val totalAccountsAmount = (accounts.indices).sumOf { accounts[it].amount }
    val totalBillsAmount = (bills.indices).sumOf { bills[it].amount }
}

Klasa `Account` zawiera pola
- `name` - nazwa konta
- `number` - numer rachunku
- `amount` - zebrana kwota
- `color` - kolor powiązany z kontem

Klasa `Bills`:
- `name` - nazwa rachunku
- `endDate` - data spłaty
- `amount` - kwota do spłaty
- `color` - kolor powiązany z rachunkiem

W obiekcie `DataProvider` znnajdują się:
- `accounts` - lista wszystkich kont
- `bills` - lista wszystkich rachunków
- `totalAccountsAmount` - całkowita kwota na wszystkich rachunkach
- `totalBillsAmount` - całkowita kwota wszystkich rachunków

### **AccountsFragment**

Ponieważ `AccountsFragment` i `BillsFragment` będą używać tego samego layoutu, zdefiniujemy jeden plik `xml` o nazwie `fragment_detail_info`, rozpocznijmy od dodania `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.AccountsFragment">
    
        <androidx.recyclerview.widget.RecyclerView
            android:id="@+id/recyclerView"
            android:layout_width="match_parent"
            android:layout_height="0dp"
            android:layout_marginTop="12dp"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toTopOf="parent" />
        
</androidx.constraintlayout.widget.ConstraintLayout>

Utwórzmy layout elementu `RecyclerView`

`recyclerview_item_view.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    android:background="@color/dark_blue_500">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="12px"
        android:layout_height="0dp"
        android:background="@color/white"
        android:layout_marginStart="8dp"
        android:layout_marginBottom="8dp"
        android:layout_marginTop="8dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/RVNumberTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:fontFamily="serif-monospace"
        android:padding="4dp"
        android:text="******1111"
        android:textColor="@color/white"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toBottomOf="@+id/RVNameTextView" />

    <TextView
        android:id="@+id/RVNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="8dp"
        android:fontFamily="sans-serif-smallcaps"
        android:padding="4dp"
        android:text="Savings"
        android:textColor="@color/white"
        android:textSize="18sp"
        android:textStyle="bold"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/RVValueTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="16dp"
        android:text="1111111 zł"
        android:textColor="@color/white"
        android:textSize="30sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Pierwszym elementem jest `View`, będzie to pasek pokazujący kolor powiązany w danym kontem.

Będziemy wyświetlać kwoty, więc dodajmy odpowiednie formatowanie, do pakietu `utils` dodajmy plik `FormatterUtil`. W nim dodajmy jedno pole zawierające `NumberFormat`

In [ ]:
val formatter: NumberFormat = DecimalFormat("#,###.##")

Następnie zaimplementujmy `AccountAdapter`

In [ ]:
class AccountAdapter : RecyclerView.Adapter<AccountAdapter.ViewHolder>(){
    inner class ViewHolder (view: View) : RecyclerView.ViewHolder(view) {
        private val nameTextView: TextView = view.findViewById(R.id.RVNameTextView)
        private val accountNumberTextView: TextView = view.findViewById(R.id.RVNumberTextView)
        private val colorBar: View = view.findViewById(R.id.RVcolorBarView)
        private val amountTextView: TextView = view.findViewById(R.id.RVValueTextView)

        fun bind(item: Account){
            nameTextView.text = item.name
            accountNumberTextView.text = item.number.replaceRange(0 until 6, "******")
            ("${formatter.format(item.amount)} zł").also { amountTextView.text = it }
            colorBar.setBackgroundColor(item.color)
        }
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ViewHolder {
        return ViewHolder(
            LayoutInflater.from(parent.context).inflate(
                R.layout.rtecyclerview_item_view, parent, false
            ))
    }

    override fun onBindViewHolder(holder: ViewHolder, position: Int) {
        val item = DataProvider.accounts[position]
        holder.bind(item)
    }

    override fun getItemCount(): Int = DataProvider.accounts.size
}

Do `AccountsFragment` dodajmy `ViewPager2` w metodzie `onViewCreated`

In [ ]:
class AccountsFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_detail_info, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        view.findViewById<RecyclerView>(R.id.recyclerView).apply {
            adapter = AccountAdapter()
            layoutManager = LinearLayoutManager(this.context)
        }
    }
}

<img src="https://media2.giphy.com/media/grRVpWcSBxxdbt727W/giphy.gif?cid=790b76116c546b317dcc43461641e6ff4d09490235131cfa&rid=giphy.gif&ct=g" width="150" />

Kolejnym elementem będzie `DonutChart` (wykorzystamy [doughnut](https://github.com/futuredapp/donut)), do zależnościach dodajemy

In [ ]:
implementation("app.futured.donut:donut:2.2.2")

do layoutu `fragment_detail_info` dodajemy `DonutProgressView` i dwa pola `TextView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.AccountsFragment">

        <TextView
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="@string/total"
            android:textColor="@color/white"
            android:textSize="26sp"
            app:layout_constraintBottom_toTopOf="@+id/totalAmountTextView"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent" />

        <TextView
            android:id="@+id/totalAmountTextView"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="100000 zł"
            android:textColor="@color/white"
            android:textSize="38sp"
            android:fontFamily="serif"
            app:layout_constraintBottom_toBottomOf="@+id/donut_viewAccount"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <app.futured.donut.DonutProgressView
            android:id="@+id/donut_viewAccount"
            android:layout_width="300dp"
            android:layout_height="300dp"
            app:donut_bgLineColor="@color/white"
            app:donut_gapAngle="20"
            app:donut_gapWidth="4"
            app:donut_animationDuration="1500"
            app:donut_strokeWidth="6dp"
            android:layout_marginTop="16dp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <androidx.recyclerview.widget.RecyclerView
            android:id="@+id/recyclerView"
            android:layout_width="match_parent"
            android:layout_height="0dp"
            android:layout_marginTop="12dp"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/donut_viewAccount" />


</androidx.constraintlayout.widget.ConstraintLayout>

W `AccountsFragment` dodajemy obsługę `DonutProgressView` - musimy przekazać dane jako listę `DonutSection`. `DonutSection` przyjmuje trzy argumenty
- `name` - nazwę
- `color` - kolor
- `amount` - wartość z przedziału 0.0 - 1.0

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    view.findViewById<RecyclerView>(R.id.recyclerView).apply {
        adapter = AccountAdapter()
        layoutManager = LinearLayoutManager(this.context)
    }

    view.findViewById<DonutProgressView>(R.id.donut_viewAccount).apply {
        val values = (0 until DataProvider.accounts.size).map { DonutSection(
            DataProvider.accounts[it].name,
            DataProvider.accounts[it].color,
            DataProvider.accounts[it].amount.toFloat() / 
                DataProvider.totalAccountsAmount.toFloat()
        ) }
        submitData(values)
    }

    view.findViewById<TextView>(R.id.totalAmountTextView).apply {
        ("${formatter.format(DataProvider.totalAccountsAmount)} zł").also { text = it }
    }
}

W efekcie otrzymamy

<img src="https://media3.giphy.com/media/V6olKtypWuObTJXxmz/giphy.gif?cid=790b761116be30abedbc5c9f9c7a309c74243ac9e1060564&rid=giphy.gif&ct=g" width="150" />

### **BillsFragment**

Do `FormatterUtil` dodajmy `DateTimeFormatter` pozwalający sformatować datę

In [ ]:
val dateFormatter: DateTimeFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd")

Przejdźmy do `BillAdapter`

In [ ]:
class BillAdapter : RecyclerView.Adapter<BillAdapter.ViewHolder>(){
    inner class ViewHolder (view: View) : RecyclerView.ViewHolder(view) {
        private val nameTextView: TextView = view.findViewById(R.id.RVNameTextView)
        private val endDateTextView: TextView = view.findViewById(R.id.RVNumberTextView)
        private val colorBar: View = view.findViewById(R.id.RVcolorBarView)
        private val amountTextView: TextView = view.findViewById(R.id.RVValueTextView)

        fun bind(item: Bill){
            nameTextView.text = item.name
            endDateTextView.text = item.endDate.format(dateFormatter)
            ("- ${formatter.format(item.amount)} zł").also { amountTextView.text = it }
            colorBar.setBackgroundColor(item.color)
        }
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ViewHolder {
        return ViewHolder(
            LayoutInflater.from(parent.context).inflate(
                R.layout.rtecyclerview_item_view, parent, false
            ))
    }

    override fun onBindViewHolder(holder: ViewHolder, position: Int) {
        val item = DataProvider.bills[position]
        holder.bind(item)
    }

    override fun getItemCount(): Int = DataProvider.bills.size
}

Korzystamy w tego samego layoutu co przy implementacji `AccountsFragment`

In [ ]:
class BillsFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_detail_info, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        view.findViewById<RecyclerView>(R.id.recyclerView).apply {
            adapter = BillAdapter()
            layoutManager = LinearLayoutManager(this.context)
        }

        view.findViewById<DonutProgressView>(R.id.donut_viewAccount).apply {
            val values = (0 until DataProvider.bills.size).map { DonutSection(
                DataProvider.bills[it].name,
                DataProvider.bills[it].color,
                DataProvider.bills[it].amount.toFloat() / DataProvider.totalBillsAmount.toFloat()
            ) }
            submitData(values)
        }

        view.findViewById<TextView>(R.id.totalAmountTextView).apply {
            ("- ${formatter.format(DataProvider.totalBillsAmount)} zł").also { text = it }
        }
    }
}

### **OverviewFragment**

Layout `OverviewFragment` będzie się opierał na `CardView`, więc musimy dodać odpowiednią zależność do aplikacji

In [ ]:
implementation "androidx.cardview:cardview:1.0.0"

Sam layout nie zawiera żadnych nowych elementów

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    android:orientation="vertical"
    tools:context=".fragments.OverviewFragment">

    <androidx.cardview.widget.CardView
        android:id="@+id/cardView0"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="16dp"
        android:layout_marginStart="16dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical">

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:text="@string/alerts"
                android:textColor="@color/white"
                android:textSize="12sp" />

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="horizontal">

                <TextView
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="16dp"
                    android:text="@string/no_alerts"
                    android:textColor="@color/white"
                    android:textSize="24sp" />

                <Button
                    android:id="@+id/seeMoreButton"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:backgroundTint="@android:color/holo_red_light"
                    android:textColor="@color/design_default_color_on_secondary"
                    android:layout_marginEnd="8dp"
                    android:text="@string/see_more" />
            </LinearLayout>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <androidx.cardview.widget.CardView
        android:id="@+id/cardView1"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="8dp"
        android:layout_marginStart="16dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical"
            tools:context=".fragments.OverviewFragment">


            <TextView
                android:id="@+id/accountNameTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="8dp"
                android:text="@string/accounts"
                android:textColor="@color/white"
                android:textSize="12sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:id="@+id/accountTotalAmountTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="4dp"
                android:gravity="center"
                android:text="11111111 zł"
                android:textColor="@color/white"
                android:textSize="36sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountNameTextView" />

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/accountsRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="0dp"
                android:layout_weight="1"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountTotalAmountTextView" />

            <Button
                android:id="@+id/accountsButton"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                android:text="@string/go_to_accounts"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountsRecyclerView"
                android:backgroundTint="@android:color/holo_red_light"
                android:textColor="@color/design_default_color_on_secondary"/>
        </LinearLayout>


    </androidx.cardview.widget.CardView>

    <androidx.cardview.widget.CardView
        android:id="@+id/cardView2"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginStart="16dp"
        android:layout_marginTop="8dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical"
            tools:context=".fragments.OverviewFragment">


            <TextView
                android:id="@+id/billsNameTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="8dp"
                android:text="@string/bills"
                android:textColor="@color/white"
                android:textSize="12sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:id="@+id/billsTotalAmountTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="4dp"
                android:gravity="center"
                android:text="- 21999.99 zł"
                android:textColor="@color/white"
                android:textSize="36sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="@id/billsNameTextView" />

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/billsRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="0dp"
                android:layout_weight="1"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/billsTotalAmountTextView" />

            <Button
                android:id="@+id/billsButton"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                android:text="@string/go_to_bills"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/billsRecyclerView"
                android:backgroundTint="@android:color/holo_red_light"
                android:textColor="@color/design_default_color_on_secondary"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>


</LinearLayout>

Do metody `onCreateView` dodajmy obsługę wszystkich elementów. Zacznijmy od `RecyclerVew`

In [ ]:
view.findViewById<RecyclerView>(R.id.accountsRecyclerView).apply {
    adapter = AccountAdapter()
    layoutManager = LinearLayoutManager(this.context)
}

view.findViewById<RecyclerView>(R.id.billsRecyclerView).apply {
    adapter = BillAdapter()
    layoutManager = LinearLayoutManager(this.context)
}

Następnie mamy dwa pola `TextView` wyświetlające wartości wszystkich kont oraz rachunków

In [ ]:
view.findViewById<TextView>(R.id.accountTotalAmountTextView).apply {
    ("${formatter.format(DataProvider.totalAccountsAmount)} zł").also { text = it }
}

view.findViewById<TextView>(R.id.billsTotalAmountTextView).apply {
    ("- ${formatter.format(DataProvider.totalBillsAmount)} zł").also { text = it }
}

Kolejno dodajmy obsługę dwóch przycisków - w tym przykładzie dodamy przejście na konkretną pozycję w `ViewPager2` - znajduje się on w głównej aktywności, więc z poziomu fragmentu nie mamy do niego dostępu. Możemy obejść ten problem wykorzystując metod `requireActivity` oraz `findViewById`.

In [ ]:
view.findViewById<Button>(R.id.accountsButton).apply {
    setOnClickListener { activity?.findViewById<ViewPager2>(R.id.viewPager)
        ?.currentItem = 1 }
}

view.findViewById<Button>(R.id.billsButton).apply {
    setOnClickListener { activity?.findViewById<ViewPager2>(R.id.viewPager)
        ?.currentItem = 2 }
}

Ostatnim elementem będzie obsługa przycisku `SEE MORE` - tutaj wyświetlimy customowy `AlertDialog`. W pierwszsym kroku zdefiniujemy styl w pliku `themes.xml`

In [ ]:
<style name="MyDialogTheme" parent="Theme.AppCompat.Light.Dialog.Alert">
    <item name="android:background">@color/dark_blue_900</item>
    <item name="android:textColor">@color/white</item>
</style>

Następnie utwórzmy layout

`alert_dialog.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:orientation="vertical"
    android:paddingLeft="20dp"
    android:paddingRight="20dp"
    android:layout_width="match_parent"
    android:background="@color/dark_blue_900"
    android:layout_height="match_parent">

    <TextView
        android:id="@+id/alertTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:textColor="@color/white"
        android:gravity="center"
        android:text="@string/no_alerts"/>
</LinearLayout>

Na koniec przechodzimy do implementacji metody `onClick`

In [ ]:
view.findViewById<Button>(R.id.seeMoreButton).apply {
    setOnClickListener {
        AlertDialog.Builder(context, R.style.MyDialogTheme)
            .setTitle(getString(R.string.alerts)) // tytuł
            .setView(layoutInflater.inflate(R.layout.alert_dialog, null)) // layout
            .setPositiveButton("OK") { _, _ -> } // domyślna implementacja przycisku wyłączającego Dialog
            .create() // utworzenie obiektu AlertDialog
            .show() // wyświetlenie AlertDialog
    }
}

Przekazanie `null` jako parametru metody `setView` generuje ostrzeżenie - w tym przykładzie pozbędziemy się go przez umieszczenie adnotacji `@SuppressLint("InflateParams")`.

In [ ]:
@SuppressLint("InflateParams")
override fun onViewCreated(view: View, savedInstanceState: Bundle?)

Możemy przetestować aplikację.

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="150" /></td></tr></table>